In [ ]:
import numpy as np
import pandas as pd
import snowflake.connector
from datetime import datetime, timedelta
import requests
import io
from scipy.optimize import linprog
import math
import os
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpStatus, value

In [ ]:
#Cargar y concatenar los archivos que existan en un solo dataframe
dataframes = []
try:
    if os.path.exists(r"C:\Users\juane\Downloads\PLAN_PANADERIA.csv"):
        df_plan_panaderia = pd.read_csv(r"C:\Users\juane\Downloads\PLAN_PANADERIA.csv")
        dataframes.append(df_plan_panaderia)
        print("PLAN_PANADERIA.csv cargado con éxito.")
    else:
        print("PLAN_PANADERIA.csv no encontrado.")

    if os.path.exists(r"C:\Users\juane\Downloads\PLAN_CONGELADOS.csv"):
        df_plan_congelados = pd.read_csv(r"C:\Users\juane\Downloads\PLAN_CONGELADOS.csv")
        dataframes.append(df_plan_congelados)
        print("PLAN_CONGELADOS.csv cargado con éxito.")
    else:
        print("PLAN_CONGELADOS.csv no encontrado.")

    if os.path.exists(r"C:\Users\juane\Downloads\PLAN_COMPRA_GENERAL.csv"):
        df_plan_compra_general = pd.read_csv(r"C:\Users\juane\Downloads\PLAN_COMPRA_GENERAL.csv")
        dataframes.append(df_plan_compra_general)
        print("PLAN_COMPRA_GENERAL.csv cargado con éxito.")
    else:
        print("PLAN_COMPRA_GENERAL.csv no encontrado.")
    
    if not dataframes:
        raise FileNotFoundError("Ningún archivo CSV de planes de compra está presente. Proceso detenido.")
    
    df_plan_externos = pd.concat(dataframes, ignore_index=True)
    df_plan_externos = df_plan_externos.reset_index(drop=True)
    print("Archivos concatenados exitosamente.")
    
except FileNotFoundError as e:
    print(e)
except Exception as e:
    print(f"Ocurrió un error inesperado: {e}")

In [ ]:
#Filtrar productos sustitutos y otros
df_plan_externos = df_plan_externos[~df_plan_externos['Sku Id'].isin([22518])]

In [ ]:
#Dataframe que relaciona warehouseId, cityId y CityName
warehouse_city_mapping = pd.DataFrame({
    'Warehouseid': [49, 63, 66, 84, 107, 21, 25, 27, 29, 31, 48, 52, 54, 55, 68, 80, 87, 91, 92, 3, 4, 5, 6, 7, 108, 109, 2, 75, 79, 90, 93, 8, 9, 11, 12, 14, 16, 17, 18, 19, 20, 23, 24, 26, 28, 30, 33, 47, 50, 51, 53, 56, 58, 60, 61, 67, 69, 70, 72, 73, 78, 110, 103, 104, 106, 105],
    'CITY_ID': [3, 3, 3, 5, 5, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 35, 35, 35, 35, 35, 35, 35, 36, 36, 36, 36, 36, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 40, 42, 42, 43],
    'CITY_NAME': ['Barranquilla', 'Barranquilla', 'Barranquilla', 'Bucaramanga', 'Bucaramanga', 'Medellín', 'Medellín', 'Medellín', 'Medellín', 'Medellín', 'Medellín', 'Medellín', 'Medellín', 'Medellín', 'Medellín', 'Medellín', 'Medellín', 'Medellín', 'Medellín', 'Cartagena', 'Barranquilla', 'Cartagena', 'Cartagena', 'Cartagena', 'Santa Marta', 'Villavicencio', 'Cali', 'Cali', 'Cali', 'Cali', 'Cali', 'Bogotá', 'Bogotá', 'Bogotá', 'Bogotá', 'Bogotá', 'Bogotá', 'Bogotá', 'Bogotá', 'Bogotá', 'Bogotá', 'Bogotá', 'Bogotá', 'Bogotá', 'Bogotá', 'Bogotá', 'Bogotá', 'Bogotá', 'Bogotá', 'Bogotá', 'Bogotá', 'Bogotá', 'Bogotá', 'Bogotá', 'Bogotá', 'Bogotá', 'Bogotá', 'Bogotá', 'Bogotá', 'Bogotá', 'Bogotá', 'Bogotá', 'Armenia', 'Pereira', 'Ibagué', 'Montería']
})

In [ ]:
#Alistar el dataframe para ser procesado
df_plan_externos["WAREHOUSE_PRODUCT_ID"]=df_plan_externos["Warehouseid"].astype(str)+"-"+df_plan_externos["Sku Id"].astype(str)
df_plan_externos["ADJUSTED_QUANTITY"] = df_plan_externos["Purchase Quantity (Min Val Adj.) Editable"]
df_plan_externos["Variable"] = df_plan_externos.apply(lambda row: LpVariable(f"Q_{row['WAREHOUSE_PRODUCT_ID']}", lowBound=row["ADJUSTED_QUANTITY"], cat="Integer"), axis=1)
df_plan_externos["Average Forecasts for Next 7 Days"] = df_plan_externos["Average Forecasts for Next 7 Days"].replace(0, 0.001)
df_plan_externos = df_plan_externos.merge(warehouse_city_mapping, on='Warehouseid', how='left')
df_plan_externos = df_plan_externos.reset_index(drop=True)

In [ ]:
#Diccionario que contiene los multiplos para proveedor y producto
multiples_restrictions = {
    # 395: { #Spiga
    #     17779: 6,
    #     17780: 6,
    #     17783: 6,
    #     17784: 6,
    #     17785: 6,
    #     17786: 6,
    #     17787: 4,
    #     17788: 6,
    #     17789: 6,
    #     17790: 6
    # },
    77:{ #Pan pa ya
        15561:5,
        15128:5,
        15130:5,
        7759:5,
        11407:5,
        7753:5,
        15560:2,
        15126:2
    },        
    204:5, #La Boutique
    168:10, #Glucloud
     4:10, #Inversiones Liev
    318:10, #Evapify
    185:10, #Relx
    392:3 #Mestri Milano
}

In [ ]:
supplier_restrictions = {
    395: [ #Spiga
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 3, 'min_value': 150000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 4, 'min_value': 150000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 5, 'min_value': 150000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 6, 'min_value': 150000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 7, 'min_value': 150000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 49, 'min_value': 150000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 63, 'min_value': 150000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 66, 'min_value': 150000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 2, 'min_value': 200000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 8, 'min_value': 200000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 9, 'min_value': 200000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 11, 'min_value': 200000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 12, 'min_value': 200000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 14, 'min_value': 200000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 17, 'min_value': 200000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 18, 'min_value': 200000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 19, 'min_value': 200000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 21, 'min_value': 200000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 24, 'min_value': 200000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 25, 'min_value': 200000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 27, 'min_value': 200000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 28, 'min_value': 200000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 29, 'min_value': 200000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 47, 'min_value': 200000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 48, 'min_value': 200000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 50, 'min_value': 200000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 51, 'min_value': 200000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 52, 'min_value': 200000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 55, 'min_value': 200000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 67, 'min_value': 200000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 78, 'min_value': 200000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 79, 'min_value': 200000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 84, 'min_value': 200000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 90, 'min_value': 200000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 93, 'min_value': 200000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 107, 'min_value': 200000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 110, 'min_value': 200000, 'max_value': 10000000}     
    ],
    63: [ #Artesa
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID':8, 'min_value': 60000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID':9, 'min_value': 60000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID':11, 'min_value': 60000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID':12, 'min_value': 60000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID':14, 'min_value': 60000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID':17, 'min_value': 60000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID':18, 'min_value': 60000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID':19, 'min_value': 60000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID':24, 'min_value': 60000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID':47, 'min_value': 60000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID':50, 'min_value': 60000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID':51, 'min_value': 60000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID':67, 'min_value': 60000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID':110, 'min_value': 60000, 'max_value': 10000000},
        {'type': 'quantity', 'level': 'product', 'SKU_ID':9109, 'min_quantity': 40, 'max_quantity': 160},
        {'type': 'quantity', 'level': 'product', 'SKU_ID':12510, 'min_quantity': 40, 'max_quantity': 160}
        
    ],
    69: [ #Deli
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID':21, 'min_value': 300000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID':48, 'min_value': 300000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID':52, 'min_value': 300000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID':68, 'min_value': 300000, 'max_value': 10000000}
    ],
    57: [ #Vive Agro
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 8, 'min_value': 250000, 'max_value': 5000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 9, 'min_value': 250000, 'max_value': 5000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 11, 'min_value': 250000, 'max_value': 5000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 12, 'min_value': 250000, 'max_value': 5000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 14, 'min_value': 250000, 'max_value': 5000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 17, 'min_value': 250000, 'max_value': 5000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 18, 'min_value': 250000, 'max_value': 5000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 19, 'min_value': 250000, 'max_value': 5000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 21, 'min_value': 250000, 'max_value': 5000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 24, 'min_value': 250000, 'max_value': 5000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 28, 'min_value': 250000, 'max_value': 5000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 29, 'min_value': 250000, 'max_value': 5000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 47, 'min_value': 250000, 'max_value': 5000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 48, 'min_value': 250000, 'max_value': 5000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 50, 'min_value': 250000, 'max_value': 5000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 51, 'min_value': 250000, 'max_value': 5000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 52, 'min_value': 250000, 'max_value': 5000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 67, 'min_value': 250000, 'max_value': 5000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 68, 'min_value': 250000, 'max_value': 5000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 78, 'min_value': 250000, 'max_value': 5000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 110, 'min_value': 250000, 'max_value': 5000000}
    ],
    365: [ #Kampos carnicos
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 2, 'min_quantity': 10, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 3, 'min_quantity': 10, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 4, 'min_quantity': 10, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 5, 'min_quantity': 10, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 6, 'min_quantity': 10, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 7, 'min_quantity': 10, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 8, 'min_quantity': 10, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 9, 'min_quantity': 10, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 11, 'min_quantity': 10, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 12, 'min_quantity': 10, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 14, 'min_quantity': 10, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 17, 'min_quantity': 10, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 18, 'min_quantity': 10, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 19, 'min_quantity': 10, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 21, 'min_quantity': 10, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 23, 'min_quantity': 10, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 24, 'min_quantity': 10, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 25, 'min_quantity': 10, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 26, 'min_quantity': 10, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 27, 'min_quantity': 10, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 28, 'min_quantity': 10, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 29, 'min_quantity': 10, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 33, 'min_quantity': 10, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 47, 'min_quantity': 10, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 48, 'min_quantity': 10, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 49, 'min_quantity': 10, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 50, 'min_quantity': 10, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 51, 'min_quantity': 10, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 52, 'min_quantity': 10, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 55, 'min_quantity': 10, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 63, 'min_quantity': 10, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 66, 'min_quantity': 10, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 67, 'min_quantity': 10, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 68, 'min_quantity': 10, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 70, 'min_quantity': 10, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 72, 'min_quantity': 10, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 73, 'min_quantity': 10, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 78, 'min_quantity': 10, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 79, 'min_quantity': 10, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 84, 'min_quantity': 10, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 90, 'min_quantity': 10, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 93, 'min_quantity': 10, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 107, 'min_quantity': 10, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 110, 'min_quantity': 10, 'max_quantity': 5000},       
    ],
    204: [ #La Boutique
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':2, 'min_quantity': 5, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':3, 'min_quantity': 5, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':4, 'min_quantity': 5, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':5, 'min_quantity': 5, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':6, 'min_quantity': 5, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':7, 'min_quantity': 5, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':8, 'min_quantity': 5, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':9, 'min_quantity': 5, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':11, 'min_quantity': 5, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':12, 'min_quantity': 5, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':14, 'min_quantity': 5, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':17, 'min_quantity': 5, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':18, 'min_quantity': 5, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':19, 'min_quantity': 5, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':21, 'min_quantity': 5, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':24, 'min_quantity': 5, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':25, 'min_quantity': 5, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':28, 'min_quantity': 5, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':29, 'min_quantity': 5, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':47, 'min_quantity': 5, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':48, 'min_quantity': 5, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':49, 'min_quantity': 5, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':50, 'min_quantity': 5, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':51, 'min_quantity': 5, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':52, 'min_quantity': 5, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':55, 'min_quantity': 5, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':63, 'min_quantity': 5, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':66, 'min_quantity': 5, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':67, 'min_quantity': 5, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':68, 'min_quantity': 5, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':78, 'min_quantity': 5, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':79, 'min_quantity': 5, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':84, 'min_quantity': 5, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':90, 'min_quantity': 5, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':107, 'min_quantity': 5, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':110, 'min_quantity': 5, 'max_quantity': 5000},
        
    ],
    385: [ #Luisa Postres
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':8, 'min_quantity': 12, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':9, 'min_quantity': 12, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':11, 'min_quantity': 12, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':12, 'min_quantity': 12, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':14, 'min_quantity': 12, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':17, 'min_quantity': 12, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':18, 'min_quantity': 12, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':19, 'min_quantity': 12, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':24, 'min_quantity': 12, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':28, 'min_quantity': 12, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':47, 'min_quantity': 12, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':50, 'min_quantity': 12, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':51, 'min_quantity': 12, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':67, 'min_quantity': 12, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':110, 'min_quantity': 12, 'max_quantity': 5000}     
    ],
    166: [ #Selvati
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 3, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 4, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 5, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 6, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 7, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 8, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 9, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 11, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 12, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 14, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 17, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 18, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 19, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 21, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 24, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 25, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 27, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 29, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 47, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 48, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 49, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 50, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 51, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 52, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 55, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 63, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 66, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 67, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 68, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 110, 'min_quantity': 20, 'max_quantity': 5000}
    ],
    386: [ #Toscana
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':8, 'min_quantity': 15, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':11, 'min_quantity': 15, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':12, 'min_quantity': 15, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':14, 'min_quantity': 15, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':17, 'min_quantity': 15, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':18, 'min_quantity': 15, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':19, 'min_quantity': 15, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':24, 'min_quantity': 15, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':28, 'min_quantity': 15, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':47, 'min_quantity': 15, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':50, 'min_quantity': 15, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':51, 'min_quantity': 15, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':67, 'min_quantity': 15, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':78, 'min_quantity': 15, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':110, 'min_quantity': 15, 'max_quantity': 5000}  
    ],
    168:[ #Glucloud
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 2, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 3, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 4, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 5, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 6, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 7, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 8, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 9, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 11, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 12, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 14, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 16, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 17, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 18, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 19, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 21, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 23, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 24, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 25, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 26, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 27, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 28, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 29, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 31, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 33, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 47, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 48, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 49, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 50, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 51, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 52, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 55, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 58, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 60, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 61, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 63, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 66, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 67, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 68, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 70, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 72, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 73, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 78, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 79, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 84, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 90, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 91, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 93, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 103, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 104, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 105, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 106, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 107, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 108, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 109, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 110, 'min_quantity': 30, 'max_quantity': 5000}
    ],
    4:[ #Inversiones Liev
        {'type': 'quantity', 'level': 'city', 'CITY_ID': 37, 'min_quantity': 250, 'max_quantity': 5000}
    ],
    185:[ #RELX
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 2, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 3, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 4, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 5, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 6, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 7, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 8, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 9, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 11, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 12, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 14, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 16, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 17, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 18, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 19, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 21, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 23, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 24, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 25, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 26, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 27, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 28, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 29, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 31, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 33, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 47, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 48, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 49, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 50, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 51, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 52, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 55, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 58, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 60, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 61, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 63, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 66, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 67, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 68, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 70, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 72, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 73, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 78, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 79, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 80, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 84, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 90, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 91, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 93, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 103, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 104, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 105, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 106, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 107, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 108, 'min_quantity': 30, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 110, 'min_quantity': 30, 'max_quantity': 5000}
    ],
    318: [ #Evapify
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 2, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 3, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 4, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 5, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 6, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 7, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 8, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 9, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 11, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 12, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 14, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 16, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 17, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 18, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 19, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 21, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 23, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 24, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 25, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 26, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 27, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 28, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 29, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 31, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 33, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 47, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 48, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 49, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 50, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 51, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 52, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 55, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 58, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 60, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 61, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 63, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 66, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 67, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 68, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 70, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 72, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 73, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 78, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 79, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 80, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 84, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 90, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 91, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 93, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 103, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 104, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 105, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 106, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 107, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 108, 'min_quantity': 20, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID': 110, 'min_quantity': 20, 'max_quantity': 5000}
    ],
    77:[ #Pan pa ya
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 2, 'min_value': 80000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 3, 'min_value': 80000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 4, 'min_value': 80000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 5, 'min_value': 80000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 6, 'min_value': 80000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 7, 'min_value': 80000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 8, 'min_value': 80000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 9, 'min_value': 80000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 11, 'min_value': 80000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 12, 'min_value': 80000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 14, 'min_value': 80000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 17, 'min_value': 80000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 18, 'min_value': 80000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 19, 'min_value': 80000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 21, 'min_value': 80000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 23, 'min_value': 80000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 24, 'min_value': 80000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 25, 'min_value': 80000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 27, 'min_value': 80000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 28, 'min_value': 80000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 29, 'min_value': 80000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 33, 'min_value': 80000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 47, 'min_value': 80000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 48, 'min_value': 80000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 49, 'min_value': 80000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 50, 'min_value': 80000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 51, 'min_value': 80000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 52, 'min_value': 80000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 55, 'min_value': 80000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 67, 'min_value': 80000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 63, 'min_value': 80000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 66, 'min_value': 80000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 68, 'min_value': 80000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 70, 'min_value': 80000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 72, 'min_value': 80000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 73, 'min_value': 80000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 78, 'min_value': 80000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 79, 'min_value': 80000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 84, 'min_value': 80000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 90, 'min_value': 80000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 93, 'min_value': 80000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 104, 'min_value': 80000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 105, 'min_value': 80000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 107, 'min_value': 80000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 108, 'min_value': 80000, 'max_value': 10000000},
        {'type': 'monetary', 'level': 'warehouse', 'WAREHOUSE_ID': 110, 'min_value': 80000, 'max_value': 10000000}
    ],
    354:[ #Sierra Viva
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':21, 'min_quantity': 12, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':25, 'min_quantity': 12, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':27, 'min_quantity': 12, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':29, 'min_quantity': 12, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':48, 'min_quantity': 12, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':52, 'min_quantity': 12, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':55, 'min_quantity': 12, 'max_quantity': 5000},
        {'type': 'quantity', 'level': 'warehouse', 'WAREHOUSE_ID':68, 'min_quantity': 12, 'max_quantity': 5000}
    ]
}

In [ ]:
#Funcion que redondea al multiplo mayor más cercano
def round_to_multiple(quantity, multiple):
    return round(quantity / multiple) * multiple

In [ ]:
#Funcion que recorre el diccionario de multiplos y filtra el dataframe de manera acorde para aplicar los multiplos que correspondan
def apply_multiples_restrictions(df, multiples_restrictions):  
    for supplier, restriction in multiples_restrictions.items():
        supplier_df = df[df['Primary Supplier Id'] == supplier]
        
        if isinstance(restriction, dict):
            #Proveedores con diferentes multiplos por producto
            for product, multiple in restriction.items():
                product_df = supplier_df[supplier_df['Sku Id'] == product]
                for idx, row in product_df.iterrows():
                    ADJUSTED_QUANTITY = round_to_multiple(row['ADJUSTED_QUANTITY'], multiple)
                    df.loc[idx, 'ADJUSTED_QUANTITY'] = ADJUSTED_QUANTITY
        else:
            #El mismo multiplo para todos los productos
            for idx, row in supplier_df.iterrows():
                ADJUSTED_QUANTITY = round_to_multiple(row['ADJUSTED_QUANTITY'], restriction)
                df.loc[idx, 'ADJUSTED_QUANTITY'] = ADJUSTED_QUANTITY
                
    return df

In [ ]:
#Funcion que optimiza las cantidades a comprar para que se ajusten a las restricciones
def optimize(df, restriction, restriction_type, objective):
    # Selección del modelo y objetivo según tipo de restricción y objetivo
    if restriction_type == 'quantity' and objective == "increment":
        model = LpProblem("Optimization_Plan", LpMinimize)
        model += lpSum([var * (1 / avg_forecast) for var, avg_forecast in zip(df["Variable"], df["Average Forecasts for Next 7 Days"])]), "Minimize_Increments"
        model += lpSum(df["Variable"]) >= restriction, "Sum_Adjusted_Quantity"
       # Restricción de vida útil para cada SKU
        for var, avg_forecast, shelf_life in zip(df["Variable"], df["Average Forecasts for Next 7 Days"], df["Shelf Life"]):
            model += (var * (1 / avg_forecast)) * 1.5 <= shelf_life, f"Shelf_Life_Constraint_{var.name}"
        model.solve()
        if LpStatus[model.status] == "Optimal":
            df["ADJUSTED_QUANTITY"] = [var.varValue for var in df["Variable"]]
    elif restriction_type == 'quantity' and objective == "decrease":
        model = LpProblem("Optimization_Plan", LpMaximize)
        model += lpSum([var * (1 / avg_forecast) for var, avg_forecast in zip(df["Variable"], df["Average Forecasts for Next 7 Days"])]), "Maximize_Increments"
        model += lpSum(df["Variable"]) <= restriction, "Sum_Adjusted_Quantity"
        model.solve()
        if LpStatus[model.status] == "Optimal":
            df["ADJUSTED_QUANTITY"] = [var.varValue for var in df["Variable"]]
    elif restriction_type == 'monetary' and objective == "increment":
        # Restricción de valor monetario con precios del proveedor
        model = LpProblem("Optimization_Plan", LpMinimize)
        model += lpSum([var * (1 / avg_forecast) for var, avg_forecast in zip(df["Variable"], df["Average Forecasts for Next 7 Days"])]), "Minimize_Increments"
        lpSum([var * price for var, price in zip(df["Variable"], df['Supplier Price Avg'])]) >= restriction, "Sum_Adjusted_Quantity"
        lpSum([var * price for var, price in zip(df["Variable"], df['Supplier Price Avg'])]) <= restriction * 1.1
        for var, avg_forecast, shelf_life in zip(df["Variable"], df["Average Forecasts for Next 7 Days"], df["Shelf Life"]):
            model += (var * (1 / avg_forecast)) * 1.5 <= shelf_life, f"Shelf_Life_Constraint_{var.name}"
        model.solve()
        if LpStatus[model.status] == "Optimal":
            df["ADJUSTED_QUANTITY"] = [var.varValue for var in df["Variable"]]
    elif restriction_type == 'monetary' and objective == "decrease":
        model = LpProblem("Optimization_Plan", LpMaximize)
        model += lpSum([var * (1 / avg_forecast) for var, avg_forecast in zip(df["Variable"], df["Average Forecasts for Next 7 Days"])]), "Maximize_Increments"
        # Restricción de valor monetario para disminuir
        lpSum([var * price for var, price in zip(df["Variable"], df['Supplier Price Avg'])]) <= restriction, "Sum_Adjusted_Quantity"
        model.solve()
        if LpStatus[model.status] == "Optimal":
            df["ADJUSTED_QUANTITY"] = [var.varValue for var in df["Variable"]]
    return df 

In [ ]:
#Levels: product,warehouse,city,total
#types: quantity, monetary
#Si en las restricciones no se especifica el nivel, se toma como total
df_plan_externos['VIOLATION'] = False
df_plan_externos['VIOLATION_DESCRIPTION'] = ''

for supplier, restrictions in supplier_restrictions.items():
    supplier_df = df_plan_externos[df_plan_externos['Primary Supplier Id'] == supplier]
    if supplier_df.empty:
        continue
    for restriction in restrictions:
        restriction_type = restriction['type']
        level = restriction['level']
        if level == 'warehouse':
            group_col = 'Warehouseid'
            restriction_key = restriction['WAREHOUSE_ID']
        elif level == 'city':
            group_col = 'CITY_ID'
            restriction_key = restriction['CITY_ID']
        elif level == 'product':
            group_col = 'Sku Id'
            restriction_key = restriction['SKU_ID']
        else: 
            group_col = None
        if group_col:
            if group_col not in supplier_df.columns:
                continue
            grouped = supplier_df.groupby(group_col)
            if not grouped.groups:
                continue
            if restriction_key in grouped.groups:
                group = grouped.get_group(restriction_key)
                if restriction_type == 'quantity':
                    total_quantity = group['ADJUSTED_QUANTITY'].sum()
                    if total_quantity < restriction['min_quantity'] or total_quantity > restriction['max_quantity']:
                        df_plan_externos.loc[group.index, 'VIOLATION'] = True
                        df_plan_externos.loc[group.index, 'VIOLATION_DESCRIPTION'] = f'{restriction_type} violation at {level} level for {group_col}={restriction_key}'
                        if total_quantity < restriction['min_quantity']:
                            min_quantity = restriction.get('min_quantity', 0)
                            adjusted_df = optimize(group, min_quantity, restriction_type, "increment")
                            df_plan_externos.loc[adjusted_df.index, 'ADJUSTED_QUANTITY']= adjusted_df['ADJUSTED_QUANTITY'].values
                        elif total_quantity > restriction['max_quantity']:
                            max_quantity = restriction.get('max_quantity', 0)
                            adjusted_df = optimize(group, max_quantity, restriction_type, "decrease")
                            df_plan_externos.loc[adjusted_df.index, 'ADJUSTED_QUANTITY']= adjusted_df['ADJUSTED_QUANTITY'].values
                elif restriction_type == 'monetary':
                    total_value = (group['ADJUSTED_QUANTITY'] * group['Supplier Price Avg']).sum()
                    if total_value < restriction['min_value'] or total_value > restriction['max_value']:
                        df_plan_externos.loc[group.index, 'VIOLATION'] = True
                        df_plan_externos.loc[group.index, 'VIOLATION_DESCRIPTION'] = f'{restriction_type} violation at {level} level for {group_col}={restriction_key}'
                        if total_value < restriction['min_value']:
                            min_value = restriction.get('min_value', 0)
                            adjusted_df = optimize(group, min_value, restriction_type, "increment")
                            df_plan_externos.loc[adjusted_df.index, 'ADJUSTED_QUANTITY']= adjusted_df['ADJUSTED_QUANTITY'].values
                        elif total_value > restriction['max_value']:
                            max_value = restriction.get('max_value', 0)
                            adjusted_df = optimize(group, max_value, restriction_type, "decrease")
                            df_plan_externos.loc[adjusted_df.index, 'ADJUSTED_QUANTITY']= adjusted_df['ADJUSTED_QUANTITY'].values
            else:
                print(f"{group_col} -> {restriction_key} is missing from the dataframe for supplier {supplier}")
                continue      
        #Si en las restricciones no se especifica el nivel, se toma como total
        else: 
            if restriction_type == 'quantity':
                total_quantity = supplier_df['ADJUSTED_QUANTITY'].sum()
                if total_quantity < restriction['min_quantity'] or total_quantity > restriction['max_quantity']:
                    df_plan_externos.loc[supplier_df.index, 'VIOLATION'] = True
                    df_plan_externos.loc[supplier_df.index, 'VIOLATION_DESCRIPTION'] = f'{restriction_type} violation at total level'
                    if total_quantity < restriction['min_quantity']:
                        min_quantity = restriction.get('min_quantity', 0)
                        adjusted_df = optimize(group, min_quantity, restriction_type, "increment")
                        df_plan_externos.loc[adjusted_df.index, 'ADJUSTED_QUANTITY']= adjusted_df['ADJUSTED_QUANTITY'].values
                    elif total_quantity > restriction['max_quantity']:
                        max_quantity = restriction.get('max_quantity', 0)
                        adjusted_df = optimize(group, max_quantity, restriction_type, "decrease")
                        df_plan_externos.loc[adjusted_df.index, 'ADJUSTED_QUANTITY']= adjusted_df['ADJUSTED_QUANTITY'].values
            elif restriction_type == 'monetary':
                total_value = (supplier_df['ADJUSTED_QUANTITY'] * supplier_df['Supplier Price Avg']).sum()
                if total_value < restriction['min_value'] or total_value > restriction['max_value']:
                    df_plan_externos.loc[supplier_df.index, 'VIOLATION'] = True
                    df_plan_externos.loc[supplier_df.index, 'VIOLATION_DESCRIPTION'] = f'{restriction_type} violation at total level'
                    if total_value < restriction['min_value']:
                            min_value = restriction.get('min_value', 0)
                            adjusted_df = optimize(group, min_value, restriction_type, "increment")
                            df_plan_externos.loc[adjusted_df.index, 'ADJUSTED_QUANTITY']= adjusted_df['ADJUSTED_QUANTITY'].values
                    elif total_value > restriction['max_value']:
                        max_value = restriction.get('max_value', 0)
                        adjusted_df = optimize(group, max_value, restriction_type, "decrease")
                        df_plan_externos.loc[adjusted_df.index, 'ADJUSTED_QUANTITY']= adjusted_df['ADJUSTED_QUANTITY'].values
                        
# Apply multiples restrictions and adjust quantities                    
df_plan_externos = apply_multiples_restrictions(df_plan_externos, multiples_restrictions)                        

In [ ]:
df_plan_externos['STATUS'] = 'Pass'
df_plan_externos['STATUS_DESCRIPTION'] = ''

for supplier, restrictions in supplier_restrictions.items():
    supplier_df = df_plan_externos[df_plan_externos['Primary Supplier Id'] == supplier]
    if supplier_df.empty:
        continue
    for restriction in restrictions:
        restriction_type = restriction['type']
        level = restriction['level']

        if level == 'warehouse':
            group_col = 'Warehouseid'
            restriction_key = restriction['WAREHOUSE_ID']
        elif level == 'city':
            group_col = 'CITY_ID'
            restriction_key = restriction['CITY_ID']
        elif level == 'product':
            group_col = 'Sku Id'
            restriction_key = restriction['SKU_ID']
        else: 
            group_col = None

        if group_col:
            if group_col not in supplier_df.columns:
                continue
            grouped = supplier_df.groupby(group_col)

            if not grouped.groups:
                continue

            if restriction_key in grouped.groups:
                group = grouped.get_group(restriction_key)
                if restriction_type == 'quantity':
                    total_quantity = group['ADJUSTED_QUANTITY'].sum()
                    if total_quantity < restriction['min_quantity'] or total_quantity > restriction['max_quantity']:
                        df_plan_externos.loc[group.index, 'STATUS'] = 'Fail'
                        df_plan_externos.loc[group.index, 'STATUS_DESCRIPTION'] = f'{restriction_type} violation at {level} level for {group_col}={restriction_key}'

                elif restriction_type == 'monetary':
                    total_value = (group['ADJUSTED_QUANTITY'] * group['Supplier Price Avg']).sum()
                    if total_value < restriction['min_value'] or total_value > restriction['max_value']:
                        df_plan_externos.loc[group.index, 'STATUS'] = 'Fail'
                        df_plan_externos.loc[group.index, 'STATUS_DESCRIPTION'] = f'{restriction_type} violation at {level} level for {group_col}={restriction_key}'
            else:
                continue
        else: 
            if restriction_type == 'quantity':
                total_quantity = supplier_df['ADJUSTED_QUANTITY'].sum()
                if total_quantity < restriction['min_quantity'] or total_quantity > restriction['max_quantity']:
                    df_plan_externos.loc[supplier_df.index, 'STATUS'] = 'Fail'
                    df_plan_externos.loc[supplier_df.index, 'STATUS_DESCRIPTION'] = f'{restriction_type} violation at total level'

            elif restriction_type == 'monetary':
                total_value = (supplier_df['ADJUSTED_QUANTITY'] * supplier_df['Supplier Price Avg']).sum()
                if total_value < restriction['min_value'] or total_value > restriction['max_value']:
                    df_plan_externos.loc[supplier_df.index, 'STATUS'] = 'Fail'
                    df_plan_externos.loc[supplier_df.index, 'STATUS_DESCRIPTION'] = f'{restriction_type} violation at total level'

In [ ]:
fecha_actual = datetime.now().strftime("%d-%m-%Y")
nombre_archivo = f"C:/Users/juane/Downloads/PLAN_EXTERNOS_COLOMBIA_{fecha_actual}.csv"
df_plan_externos.to_csv(nombre_archivo, index=False)

In [ ]:
df_wms_input_compra_general = pd.DataFrame()
df_wms_input_compra_general['WAREHOUSE_DESTINATION'] = df_plan_externos.loc[df_plan_externos['STATUS']== 'Pass', 'Warehouseid']
df_wms_input_compra_general['SUPPLIER_ID'] = df_plan_externos.loc[df_plan_externos['STATUS']== 'Pass', 'Primary Supplier Id']
df_wms_input_compra_general['RETAIL_ID'] = df_plan_externos.loc[df_plan_externos['STATUS']== 'Pass','Sku Id']
df_wms_input_compra_general['QUANTITY'] = df_plan_externos.loc[df_plan_externos['STATUS'] == 'Pass', 'ADJUSTED_QUANTITY'].astype(int).values
df_wms_input_compra_general['PLANNED_DATE'] = pd.to_datetime(datetime.now().date()) + pd.to_timedelta(df_plan_externos.loc[df_plan_externos['STATUS']== 'Pass','Supplier Lead Time (Days)'], unit='d')
df_wms_input_compra_general['PLANNED_DATE'] = df_wms_input_compra_general['PLANNED_DATE'].dt.strftime('%d-%m-%Y')
df_wms_input_compra_general = df_wms_input_compra_general.loc[df_wms_input_compra_general['QUANTITY'] != 0].copy()
df_wms_input_compra_general = df_wms_input_compra_general.reset_index(drop=True)

In [ ]:
nombre_archivo = f"C:/Users/juane/Downloads/WMS_INPUT_PURCHASE_PLAN_{fecha_actual}.csv"
df_wms_input_compra_general.to_csv(nombre_archivo,index=False)

In [ ]:
for supplier_id in df_wms_input_compra_general['SUPPLIER_ID'].unique():
    supplier_df = df_wms_input_compra_general[df_wms_input_compra_general['SUPPLIER_ID'] == supplier_id]
    
    file_name = f"C:/Users/juane/Downloads/WMS_INPUT_PURCHASE_PLAN_{supplier_id}.csv"
    supplier_df.to_csv(file_name, index=False)